## Assignment 3
### Massimiliano Toffoli (5937930)

## Part 1
### Question 1.1

Write a function (named `msd`) that calculates the mean square displacement of the atoms. It shall read the `"trajectory.xyz"` file and calculate the part within the ⟨ ⟩:

$$
D^{\mathrm{MD}} = \frac{1}{6Nt} \left\langle \sum_{i=1}^{N} \left( \vec{r}_j(t_0 + t) - \vec{r}_j(t_0) \right)^2 \right\rangle
$$

where \( D^{\mathrm{MD}} \) is the self-diffusion coefficient for this system size, \( N \) the number of particles, and \( \vec{r}_j \) indicates the atomic position of the atom \( j \). You can choose between two different algorithms: the **single window algorithm** (using a single time origin, see algorithm 1) and the **multi window algorithm** (using many time origins, see algorithm 2).

Although the single window algorithm is easier to code, it will require much longer simulation times than a multiple window algorithm. We do advise you to implement the more advanced algorithm (without making it mandatory) as it has significant benefits compared to the single window algorithm.

---

**Algorithm 1**: Single window self-diffusion algorithm

---

```python
# Startup
MSD(t) = 0.0  # set all MSD's to 0

# Set single origin
for 0 ≤ i < npart do
    x0(i) = r(0, i)  # original location of all atoms
end for

# Start sampling all time steps
for 0 ≤ t < tmax do
    Δr² = 0

    # Every particle
    for 0 ≤ i < npart do
        Δx⃗ = r(t, i) - x0(i)       # Cartesian distance, do not adjust for PBC
        Δr² += Δx⃗ · Δx⃗            # squared radial distance between current and original position
    end for

    MSD(t) = Δr²  # get mean MSD for time difference t
end for
```
Show a plot of the average MSD per time shift (the part between ⟨⟩), where the MSD and time axis are shown in a logarithmic scale.
Indicate in the caption how you would calculate the self-diffusion coefficient from this graph.

---
**Algorithm 2**: Multiple window self-diffusion algorithm, inspired by *Understanding Molecular Simulation* by Daan Frenkel and Berend Smit.

---

```python
# Startup
Origin_Frequency = 50      # set origin storage frequency (50 is just an example)
Max_Correlations = 2500    # set maximum stored origins per particle
tmax = 400000

time0 = 0.0 * # set array with times of time origins to 0 (should have length equal to Max_Correlations)
sample_counter = 0 * # set array with times of time origins to 0 (should have length equal to Max_Correlations)
MSD(t) = 0.0 # set all MSD's to 0 (should have length equal to Max_Correlations)
t0 = 0 # counter for origin amount

# Sample AND set origins on the fly
for 0 ≤ t < tmax do
    # Add new origins
    if t % Origin_Frequency == 0 then
        tt0 = t0 % Max_Correlations
        t0 += 1 # update newest origin
        time0(tt0) = t
        # Set origin for this step
        for 1 ≤ i ≤ npart do
            x0(i) = r(0, i) # original location of all atoms
        end for
    end if

    # Sampling part
    for 0 ≤ dt < min(t0, Max_Correlations) do
        corr_time = t - time0(dt) + 1 # time difference
        if corr_time < tmax then
            sample_counter(corr_time) += 1
            Δr² = 0
            # every particle
            for 0 ≤ i < npart do
                Δx⃗ = r(t, i) - x0(i) # Cartesian distance, do not adjust for pbc
                Δr² += Δx⃗ · Δx⃗     # squared radial distance between current and original position
            end for
            MSD(corr_time) = Δr² # get mean MSD for time difference t
        end if
    end for
end for

# Final averaging
for 1 ≤ corr_time ≤ do
    MSD(corr_time) = MSD(corr_time) / sample_counter(corr_time)
end for
```
---



In [ ]:
#Massimiliano Toffoli 
# call functions here

### Question 1.2

Write a function (named `diffusionEinstein`) that calculates the self-diffusion coefficient from the output of the `msd` function. Use the method you described in the caption of your MSD result in question 1.1.


## Assignment Part 2: Finite size effects

As discussed in the class, some properties suffer from so-called finite-size effects due to hydrodynamic effects combined with the use of periodic boundary conditions. In this part of the assignment, you will investigate the finite size effects of self-diffusion coefficient.

### Question 2.1

Use the code of your MD assignment to simulate CH₄ with your thermostat. Run the simulation for at least three different simulation sizes: 362, 750 and 1500 CH₄ molecules, use the simulation settings shown in table 3. Note that these computations can take a long time, so start them at a convenient moment.


### Question 2.2

Plot your estimated self-diffusion coefficient as a function of the box size.  
Put $1/L$ (or $1/N^{1/3}$) on the x-axis and the self-diffusion coefficient on the y-axis.

---

**Algorithm 3**: Simulation parameters

---

```python
# Simulation parameters
Nfreq_pos = 10       # sample frequency position output
steps = 100000       # minimum length of simulation run [fs] (may be longer depending on your algorithm)
dt = 1               # timestep [fs]

# CH₄ methane
sigma = 3.72         # [angstrom]
epsilon = 148 * k_B  # [K]
mass = 16.04         # [g/mol]

# Phase point
rho = 358.4          # [kg/m^3] density
T = 150              # [K]
L = ...              # Compute yourself
nPart = 362..1500    # Number of atoms

```

---

### Question 2.3

Create another function (named `correctionYehHammer`) that computes the finite size correction ($D_i^{\text{corr}}$)  
(given experimental viscosity $\eta$) (During the lecture, we explicitly said that in such computations the viscosity should also be the one computed from MD. Here you can you the experimental data (e.g., from NIST database) for simplicity) and adds this to the already computed self-diffusion coefficient ($D_i^{\text{MD}}$):

$$
D_i^{\infty} = D_i^{\text{MD}} + D_i^{\text{corr}} = D^{\text{MD}} + \frac{k_B T \, \xi}{6 \pi \eta L}
$$

The box size is indicated by $L$ and the dimensionless constant for cubic lattices $\xi = 2.837298$.  
Plot the corrected self-diffusion coefficient as a function of $1/L$.


## Assignment Part 3: Production of transport properties

Use your MD code (from assignment 2) to produce long trajectories for different temperatures (see below Algorithm 4 for detailed settings).  
Compute the self-diffusion coefficients of methane using your own trajectories.  
Plot your finite size corrected results and compare with experimental results.

---

**Algorithm 4**: Simulation parameters

---

```python
# Simulation parameters
Nfreq_pos = 10       # sample frequency position output
steps = 250000       # minimum length of simulation run [fs] (may be longer depending on your algorithm)
dt = 1               # timestep [fs]

# CH₄ methane
sigma = 3.72         # [angstrom]
epsilon = 148 * k_B  # [K]
mass = 16.04         # [g/mol]

# Phase point
rho = 358.4          # [kg/m^3] density
T = [200, 300, 400]  # [K]
L = ...              # Compute yourself
nPart = 362          # Number of atoms
```
---